# Bài tập lập trình: SGD với dữ liệu thực tế

```
ProtonX - TensorFlow Class 
```

### Hướng dẫn làm bài 
- Trong bài tập này bạn sẽ sử dụng Python 3.


### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**


Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO` và thay thế các vị trí `None`.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó. 

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Phải có log quá trình training
* Mean Square Error nhỏ hơn hoặc bằng 2535885073

In [156]:
import tensorflow as tf
import pandas as pd

In [157]:
train_df = pd.read_csv('./Data/california_housing_train.csv')
val_df = pd.read_csv('./Data/california_housing_test.csv') 

In [158]:
train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


### Chuẩn hóa dữ liệu

In [159]:
X_train = train_df.drop('median_house_value', axis = 1)
y_train = train_df['median_house_value']

In [160]:
y_train

0         66900.0
1         80100.0
2         85700.0
3         73400.0
4         65500.0
           ...   
16995    111400.0
16996     79000.0
16997    103600.0
16998     85800.0
16999     94600.0
Name: median_house_value, Length: 17000, dtype: float64

In [161]:
import pandas as pd
from sklearn import preprocessing

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

column_names_to_normalize = ['longitude', 'latitude', 'total_rooms', 'total_bedrooms', 'population', 'households']

x = X_train[column_names_to_normalize].values

x_scaled = min_max_scaler.fit_transform(x)

normalized_features = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = X_train.index)

X_train[column_names_to_normalize] = normalized_features

In [162]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()

column_names_to_standardize = ['housing_median_age', 'median_income']

x = X_train[column_names_to_standardize].values

x_scaled = standard_scaler.fit_transform(x)

standardized_features = pd.DataFrame(x_scaled, columns=column_names_to_standardize, index = X_train.index)

X_train[column_names_to_standardize] = standardized_features

In [163]:
X_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,1.000000,0.175345,-1.079671,0.147885,0.198945,0.028364,0.077454,-1.252543
1,0.984064,0.197662,-0.761872,0.201608,0.294848,0.031559,0.075974,-1.081483
2,0.975100,0.122210,-0.920772,0.018927,0.026847,0.009249,0.019076,-1.170105
3,0.974104,0.116897,-1.159121,0.039515,0.052142,0.014350,0.037000,-0.362600
4,0.974104,0.109458,-0.682422,0.038276,0.050435,0.017405,0.042921,-1.026454
...,...,...,...,...,...,...,...,...
16995,0.008964,0.854410,1.859971,0.058389,0.060987,0.025337,0.060516,-0.799999
16996,0.007968,0.866100,0.588774,0.061869,0.081782,0.033381,0.076303,-0.715727
16997,0.004980,0.988310,-0.920772,0.070515,0.082247,0.034782,0.074823,-0.446663
16998,0.004980,0.984060,-0.761872,0.070384,0.085506,0.036296,0.078441,-0.997787


### Tiến hành training

In [164]:
import numpy as np
m, n = X_train.shape
ones = np.ones((m, 1))
X_train = np.concatenate((X_train, ones), axis=1)


In [165]:
X_train

array([[ 1.        ,  0.17534538, -1.07967114, ...,  0.07745437,
        -1.25254316,  1.        ],
       [ 0.98406375,  0.19766206, -0.76187201, ...,  0.07597435,
        -1.08148298,  1.        ],
       [ 0.9750996 ,  0.12221041, -0.92077158, ...,  0.01907581,
        -1.17010515,  1.        ],
       ...,
       [ 0.00498008,  0.98831031, -0.92077158, ...,  0.07482322,
        -0.44666313,  1.        ],
       [ 0.00498008,  0.98405951, -0.76187201, ...,  0.07844105,
        -0.99778717,  1.        ],
       [ 0.        ,  0.8501594 ,  1.85997083, ...,  0.04423615,
        -0.45536288,  1.        ]])

In [166]:
# Tạo tham số theta (9, ) -> (9, 1)
theta = np.random.normal(size=X_train.shape[1]).reshape((X_train.shape[1], 1))

X_train.dot(theta).shape

# Chuyển chiều từ (17000, ) -> (17000, 1)
y_train = y_train.values.reshape(-1, 1)


In [167]:
X_train.shape, theta.shape, y_train.shape

((17000, 9), (9, 1), (17000, 1))

### Xây dựng mô hình SGD tại đây

In [168]:
from sklearn.utils import shuffle

learning_rate = 0.01

def hypo(X, theta):
  # TODO 1: Tính toán hàm giả thiết
  return X.T.dot(theta)

def cost(X, y, theta, m):
  # TODO 2: Tính giá trị mất mát trên toàn bộ tập dữ liệu

  return 1/(2 * m) * np.sum((X.dot(theta)-y)**2)

def SGD(X, y, theta, learning_rate, epochs):
  m = len(X)
  for i in range(epochs):
    # TODO 3: Xóa dữ liệu
    X,y = shuffle(X,y)
    # TODO 4: Cập nhật bộ tham số
    for k in range(m):
      error = hypo(X[k], theta)- y[k]
      for j in range(len(theta) ):
        theta[j] = theta[j] - learning_rate * error *X[k][j]
  

     
    


### Tiến hành Training

In [169]:
epochs = 500
learning_rate = 0.07
SGD(X_train, y_train, theta, learning_rate, epochs)

Kiểm tra Model hiện tại với tập test

In [190]:
test_df = pd.read_csv('./Data/california_housing_test.csv') 
X_test = test_df.drop('median_house_value', axis = 1)
y_test = test_df['median_house_value']

# TODO 5: Lấy các cột cần chuẩn hóa
x_test_cols = X_test[column_names_to_normalize].values
# TODO 6: Sử dụng min và max của X_train để chuẩn hóa cho X_test

x_test_cols_scaled = min_max_scaler.fit_transform(x_test_cols)

# TODO 7: Tạo dataframe mới với các cột này
normalized_test_features = pd.DataFrame(x_test_cols_scaled, columns=column_names_to_normalize, index=X_test.index)

# TODO 8: Gán lại cho bảng ban đầu
X_test[column_names_to_normalize] = normalized_features

In [171]:
from sklearn.preprocessing import StandardScaler

column_names_to_standardize = ['housing_median_age', 'median_income']

# TODO 9: Chuẩn hóa các cột về phân phối chuẩn đơn vị
x_test_cols = X_test[column_names_to_standardize].values
x_test_cols_scaled = standard_scaler.fit_transform(x_test_cols)
standardized_test_features = pd.DataFrame(x_test_cols_scaled,columns=column_names_to_standardize, index= X_test.index)
X_test[column_names_to_standardize] = standardized_test_features

In [172]:
len(X_test)

3000

In [173]:
X_test.shape

(3000, 8)

In [188]:
# Tạo tham số theta (9, ) -> (9, 1)
theta = np.random.normal(size=X_test.shape[1]).reshape((X_test.shape[1], 1))

X_test.dot(theta).shape

# Chuyển chiều từ (17000, ) -> (17000, 1)
# y_test = y_test.values.reshape(-1, 1)

(3000, 1)

In [191]:
y_test = y_test.values.reshape(-1, 1)

In [192]:
y_test.shape

(3000, 1)

In [193]:
X_test.dot(theta).shape

(3000, 1)

# TODO 10: Tính mất mát trên bộ test
X_test = x_test_cols_scaled
y_test = y_test
cost(X_test, y_test, theta, len(X_test))

In [194]:
# TODO 10: Tính mất mát trên bộ test
X_test = X_test
y_test = y_test
cost(X_test, y_test, theta, len(X_test))

0    2.758173e+10
dtype: float64

Kết quả mong đợi: mất mát nhỏ hơn hoặc bằng `2535885073`